In [ ]:
import os
os.getcwd()

'/content/drive/My Drive/Colab/23_DL_basics/0616'

In [ ]:
%cd /content/drive/MyDrive/Colab/23_DL_basics/0616/

/content/drive/MyDrive/Colab/23_DL_basics/0616


In [ ]:
!ls

In [ ]:
import os
os.getcwd()

'/content/drive/MyDrive/Colab/23_DL_basics/0616'

In [ ]:
!unzip -qq "/content/drive/MyDrive/Colab/23_DL_basics/0616/data.zip"

In [ ]:
!ls

In [ ]:
!mkdir data

In [ ]:
!ls

In [ ]:
!mv train/ data/

In [ ]:
!mv valid/ data/

In [ ]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras

import argparse

In [ ]:
train_set_dir = os.path.join(os.getcwd(), 'data'+ '/train')
valid_set_dir = os.path.join(os.getcwd(), 'data'+'/valid')

# image data generator
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                      rotation_range=15,
                                                      width_shift_range=0.1,
                                                      height_shift_range=0.1,
                                                      shear_range=0.2,
                                                      zoom_range=0.2,
                                                      horizontal_flip=True,
                                                      fill_mode='nearest')

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


# flow from directory
train_generator = train_datagen.flow_from_directory(train_set_dir,
                                                target_size=(224,224),
                                                classes=['Cats', 'NonCats'],
                                                batch_size=32,
                                                shuffle=True,
                                                seed=168369,
                                                class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_set_dir,
                                                target_size=(224,224),
                                                classes=['Cats', 'NonCats'],
                                                batch_size=32,
                                                class_mode='binary')
train_step = train_generator.n // 32
valid_step = valid_generator.n // 32

In [ ]:
args = easydict.EasyDict({
    "lr" : 1e-5,
    "epoch" : 40,
    "ver" : "11",
    "trial" : '1'
})

In [ ]:
with tf.device('/device:GPU:0'):
    vgg_base = tf.keras.applications.VGG16(include_top=False, input_shape=(224, 224, 3), weights="imagenet")

    # trainable = False
    # for layer in vgg_base.layers:
    #     if layer.name == 'block5_conv2':
    #         trainable = True
    #     layer.trainable = trainable
    # print(vgg_base.summary())

    for layer in vgg_base.layers[:15]:
        layer.trainable = False
    for layer in vgg_base.layers[15:]:
        layer.trainable = True

    if args.ver=='0':
        for layer in vgg_base.layers:
            layer.trainable = False
    if args.ver=='1':
        vgg_model = tf.keras.models.Sequential([
            vgg_base,
            tf.keras.layers.GlobalMaxPooling2D(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
    elif args.ver=='2':
        vgg_model = tf.keras.models.Sequential([
            vgg_base,
            tf.keras.layers.GlobalMaxPooling2D(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
    elif args.ver == '3':
        vgg_model = tf.keras.models.Sequential([
            vgg_base,
            tf.keras.layers.GlobalMaxPooling2D(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
    elif args.ver=='4':
        vgg_model = tf.keras.models.Sequential([
            vgg_base,
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
    elif args.ver=='5':
        vgg_model = tf.keras.models.Sequential([
            vgg_base,
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
    elif args.ver=='6':
        vgg_model = tf.keras.models.Sequential([
            vgg_base,
            tf.keras.layers.GlobalMaxPooling2D(),
            tf.keras.layers.Dense(1024, activation='relu'),
            tf.keras.layers.Dropout(0.5),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
    elif args.ver=='7'or args.ver=='8' or args.ver=='9':
        vgg_model = tf.keras.models.Sequential([
            vgg_base,
            tf.keras.layers.GlobalMaxPooling2D(),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])


    METRICS = [
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.BinaryAccuracy(name='accuracy')
    ]

    if args.ver=='9':
        vgg_model.compile(optimizer=tf.keras.optimizers.RMSprop(args.lr),
                    loss='binary_crossentropy',
                    metrics=METRICS
                    )
    else:
        vgg_model.compile(optimizer=tf.keras.optimizers.Adam(args.lr),
                    loss='binary_crossentropy',
                    metrics=METRICS
                    )


    checkpoint_file_path = 'checkpoint/' + str(args.trial)
    if not os.path.exists(os.path.join(os.getcwd(), checkpoint_file_path)):
        os.makedirs(os.path.join(os.getcwd(), checkpoint_file_path))

    # es = EarlyStopping(monitor='val_loss', mode='min', patience=20, restore_best_weights=True, verbose=1)
    ch = ModelCheckpoint(checkpoint_file_path+"/vgg16_{epoch:02d}-{val_loss:.2f}.h5", monitor='val_loss', mode='min', save_best_only=False, initial_value_threshold=0.061)

    history = vgg_model.fit_generator(train_generator,
                                    steps_per_epoch=train_step,
                                    epochs=args.epoch,
                                    validation_data=valid_generator,
                                    validation_steps=valid_step,
                                    callbacks = [ch])
    vgg_model.save('models/vgg_'+str(args.trial)+'_model_epoch_30_1e-5.h5')

In [ ]:
def training(args):

    train_set_dir = os.path.join(os.getcwd(), 'data'+ '/train')
    valid_set_dir = os.path.join(os.getcwd(), 'data'+'/valid')

    # image data generator
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                            rotation_range=15,
                                                            width_shift_range=0.1,
                                                            height_shift_range=0.1,
                                                            shear_range=0.2,
                                                            zoom_range=0.2,
                                                            horizontal_flip=True,
                                                            fill_mode='nearest')

    valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


    # flow from directory
    train_generator = train_datagen.flow_from_directory(train_set_dir,
                                                      target_size=(224,224),
                                                      classes=['Cats', 'NonCats'],
                                                      batch_size=32,
                                                      shuffle=True,
                                                      seed=168369,
                                                      class_mode='binary')
    valid_generator = valid_datagen.flow_from_directory(valid_set_dir,
                                                      target_size=(224,224),
                                                      classes=['Cats', 'NonCats'],
                                                      batch_size=32,
                                                      class_mode='binary')
    train_step = train_generator.n // 32
    valid_step = valid_generator.n // 32

    with tf.device('/device:GPU:0'):
        vgg_base = tf.keras.applications.VGG16(include_top=False, input_shape=(224, 224, 3), weights="imagenet")

        # trainable = False
        # for layer in vgg_base.layers:
        #     if layer.name == 'block5_conv2':
        #         trainable = True
        #     layer.trainable = trainable
        # print(vgg_base.summary())

        for layer in vgg_base.layers[:15]:
            layer.trainable = False
        for layer in vgg_base.layers[15:]:
            layer.trainable = True

        if args.ver=='0':
            for layer in vgg_base.layers:
                layer.trainable = False
        if args.ver=='1':
            vgg_model = tf.keras.models.Sequential([
                vgg_base,
                tf.keras.layers.GlobalMaxPooling2D(),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(512, activation='relu'),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(1, activation='sigmoid')
            ])
        elif args.ver=='2':
            vgg_model = tf.keras.models.Sequential([
                vgg_base,
                tf.keras.layers.GlobalMaxPooling2D(),
                tf.keras.layers.Dense(512, activation='relu'),
                tf.keras.layers.Dropout(0.2),
                tf.keras.layers.Dense(1, activation='sigmoid')
            ])
        elif args.ver == '3':
            vgg_model = tf.keras.models.Sequential([
                vgg_base,
                tf.keras.layers.GlobalMaxPooling2D(),
                tf.keras.layers.Dense(512, activation='relu'),
                tf.keras.layers.Dense(1, activation='sigmoid')
            ])
        elif args.ver=='4':
            vgg_model = tf.keras.models.Sequential([
                vgg_base,
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(512, activation='relu'),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(1, activation='sigmoid')
            ])
        elif args.ver=='5':
            vgg_model = tf.keras.models.Sequential([
                vgg_base,
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(512, activation='relu'),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(1, activation='sigmoid')
            ])
        elif args.ver=='6':
            vgg_model = tf.keras.models.Sequential([
                vgg_base,
                tf.keras.layers.GlobalMaxPooling2D(),
                tf.keras.layers.Dense(1024, activation='relu'),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(1, activation='sigmoid')
            ])
        elif args.ver=='7'or args.ver=='8' or args.ver=='9':
            vgg_model = tf.keras.models.Sequential([
                vgg_base,
                tf.keras.layers.GlobalMaxPooling2D(),
                tf.keras.layers.Dense(512, activation='relu'),
                tf.keras.layers.Dropout(0.2),
                tf.keras.layers.Dense(1, activation='sigmoid')
            ])


        METRICS = [
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.BinaryAccuracy(name='accuracy')
        ]

        if args.ver=='9':
            vgg_model.compile(optimizer=tf.keras.optimizers.RMSprop(args.lr),
                        loss='binary_crossentropy',
                        metrics=METRICS
                        )
        else:
            vgg_model.compile(optimizer=tf.keras.optimizers.Adam(args.lr),
                        loss='binary_crossentropy',
                        metrics=METRICS
                        )


        checkpoint_file_path = 'checkpoint/' + str(args.trial)
        if not os.path.exists(os.path.join(os.getcwd(), checkpoint_file_path)):
            os.makedirs(os.path.join(os.getcwd(), checkpoint_file_path))

        # es = EarlyStopping(monitor='val_loss', mode='min', patience=20, restore_best_weights=True, verbose=1)
        ch = ModelCheckpoint(checkpoint_file_path+"/vgg16_{epoch:02d}-{val_loss:.2f}.h5", monitor='val_loss', mode='min', save_best_only=False, initial_value_threshold=0.061)

        history = vgg_model.fit_generator(train_generator,
                                        steps_per_epoch=train_step,
                                        epochs=args.epoch,
                                        validation_data=valid_generator,
                                        validation_steps=valid_step,
                                        callbacks = [ch])
        vgg_model.save('models/vgg_{}_model_epoch_{}_{}.h5'.format(args.trial, args.epoch, args, args.lr))

In [ ]:
import easydict

for i in ['1', '2', '3', '4', '5']:
  args = easydict.EasyDict({
      "lr" : 1e-5,
      "epoch" : 40,
      "ver" : "11",
      "trial" : i
  })

  training(args)